# Day 4: Repose Record

https://adventofcode.com/2018/day/4

## Parse input

In [1]:
import re
from datetime import datetime

# First, parse input to a list
input = []
inputre = re.compile('\[(.+)\] (.+)')

with open('inputs/4.txt') as f:
    for l in f.readlines():
        matches = inputre.findall(l)[0] # Output tuple: (time, action)
        input.append((datetime.strptime(matches[0], '%Y-%m-%d %H:%M'), matches[1]))

# Sort by time
input = sorted(input, key = lambda x: x[0])

## Process the schedule

In [2]:
import datetime

# Guard class - used to collect cumulative data for specific guards
# (used to calculate answers)
class Guard:
    def __init__(self, id):
        self.id = id
        self.napStart = None
        self.sleepTime = datetime.timedelta(0)
        self.minutes = [0]*60
        
    def sleep(self, time):
        self.napStart = time
        
    def wakeup(self, time):
        napLength = (time - self.napStart)
        self.sleepTime += napLength
        
        # Update slept minutes
        for minute in range(int(napLength.total_seconds() / 60)):
            self.minutes[(self.napStart.minute + minute) % 60] += 1            
        return (self.napStart, time)
        
    def getMostSleptMinute(self):        
        return self.minutes.index(max(self.minutes))
    
    def getTotalSleepTime(self):
        return int(self.sleepTime.total_seconds() / 60)
    
    def getMostFreqSleptMinute(self):
        return max(self.minutes)
        
# Shift class - contains nap data for one shift
# (used for visuals later)
class Shift:
    def __init__(self, guardId, startTime):
        self.naps = []
        self.guardId = guardId
        self.startTime = startTime
        # Some shifts start before midnight, but time before midnight is not
        # relevant for the question in the puzzle, so move up the start time to midnight.
        if self.startTime.hour != 0:
            self.startTime += datetime.timedelta(0, 60 * (60 - self.startTime.minute))
    def addNap(self, napTimes):
        self.naps.append(napTimes)
        
guards = {}
shifts = []

# Process the actions
reguard = re.compile('Guard #(\d+) begins shift')
guardOnShift = None
for time, action in input:
    if action == "falls asleep":
        guardOnShift.sleep(time)
    elif action == "wakes up":
        shifts[-1].addNap(guardOnShift.wakeup(time))
    else: # Guard begins shift
        guardId = int(reguard.findall(action)[0])
        if guardId not in guards.keys():
            guards[guardId] = Guard(guardId)
        guardOnShift = guards[guardId]
        shifts.append(Shift(guardId, time))

## Answer #1
Strategy 1: Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

In [3]:
answer1Guard = max(guards.values(), key = lambda x: x.getTotalSleepTime())
answer1Minute = answer1Guard.getMostSleptMinute()
answer1 = (answer1Guard.id * answer1Minute)
print("Answer #1: %d (guard: %d, minute: %d)" % (answer1, answer1Guard.id, answer1Minute))

Answer #1: 87681 (guard: 2657, minute: 33)


## Answer #2
Strategy 2: Of all guards, which guard is most frequently asleep on the same minute?

In [4]:
answer2Guard = max(guards.values(), key = lambda x: x.getMostFreqSleptMinute())
answer2Minute = answer2Guard.getMostSleptMinute()
answer2 = (answer2Guard.id * answer2Minute)
print("Answer #2: %d (guard: %d, minute: %d)" % (answer2, answer2Guard.id, answer2Minute))

Answer #2: 136461 (guard: 3499, minute: 39)


## Shifts visualised

One cell is one minute. Horizontal axis: time (60 minutes), vertical axis: shifts

Cell colors:
* Black - No active shift
* Blue - Active guard shift
* Red - Guard sleeping
* Yellow - Answer #1
* Purple - Answer #2

In [5]:
import ipythonblocks

grid = ipythonblocks.BlockGrid(60, len(shifts), block_size = 10)

for day, shift in enumerate(shifts):
    grid[day, shift.startTime.minute:60] = (0, 0, 150)
    for nap in shift.naps:
        grid[day, nap[0].minute:nap[1].minute] = (150, 0, 0)
    # Highlight answers
    if shift.guardId == answer1Guard.id:
        grid[day, answer1Minute] = (255, 255, 0)
    if shift.guardId == answer2Guard.id:
        grid[day, answer2Minute] = (255, 0, 255)
grid

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
